# Keras Intensive Run

Instructions
1. Define the global variables;
2. Run the second snippet. It builds the network and saves the output to a folder displayed on the console.

In [1]:
# TEST VARIABLES
labelsGroup = [
    ["happy", "sad"],
]
iterationsGroup = [100]
hiddenLayersGroup = [
    [144],
]
batchSize = 128
hyperParametersGroup = [0, 0.01]

In [9]:
# GLOBAL VARIABLES
labelsGroup = [
    # Gonçalo
    ["happy", "neutral"],
    ["angry", "fear", "happy", "neutral", "sad", "surprise"],
    ["fear", "angry"],
    ["happy", "sad"],
    # Margarida
    # ["fear", "sad", "happy"],
    # ["happy", "angry", "neutral"],
    # ["fear", "angry", "surprise"],
    # ["fear", "angry", "surprise", "neutral"],
    
]
iterationsGroup = [200]
hiddenLayersGroup = [
    [20],
    [144],
    [144, 144],
    [300,200,100,50],
    [300,100,100,100],
]
batchSize = 128
hyperParametersGroup = [0, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]

In [10]:
def confusionMatrix(labels, pred, y):
    
    confusionTFPN = {}
    confusionLabels = {}
    
    # Foreach emotion
    for ie in range(len(labels)):
        # TRUE AND FALSE POSITIVES AND NEGATIVES (TFPN)
        
        # Get indexes where emotion was predicted
        ieIndexesPred = [i for i in range(pred.size) if pred[i]==ie]
        ieIndexesNotPred = [i for i in range(pred.size) if pred[i]!=ie]

        # Predicted, and Actual
        TP = sum(pred[:,np.newaxis][ieIndexesPred]==y[ieIndexesPred])
        # Predicted, but not actual
        FP = sum(pred[:,np.newaxis][ieIndexesPred]!=y[ieIndexesPred])
        # Not predicted, but actual
        FN = sum(y[ieIndexesNotPred]==ie)
        # Not predicted and not actual
        TN = sum(y[ieIndexesNotPred]!=ie)

        TP = TP[0] if TP else 0
        FP = FP[0] if FP else 0
        FN = FN[0] if FN else 0
        TN = TN[0] if TN else 0
        
        confusionTFPN[labels[ie]] = {
            'TP': TP,
            'FP': FP,
            'FN': FN,
            'TN': TN
        }
        
        # CONFUSION WITH OTHER labels (Confusion)
        # For emotion e
        # Count the number of predictions made for all classes
        # Foreach emotion, check how many times it has been predicted 
        
        # Get indexes where emotion is real
        ieIndexesY = [i for i in range(y.size) if y[i]==ie]
        
        # Foreach matching prediction, check what emotion was predicted
        confusionLabels[labels[ie]] = {
            e: sum(pred[:,np.newaxis][ieIndexesY]==labels.index(e))[0] for e in labels
        }
        
    return confusionTFPN, confusionLabels

def outputConfusionTables(labels, testExamplesNumber, confusionTFPN, confusionLabels, axs, axsLine, axsTitle):
     # TABLES
    rows = tuple(labels)
    
    # TABLES / TFPN
    # Output confusion matrix as plot table
    data = [
        [
            f"{vals['TP']} ({vals['TP']/testExamplesNumber*100:.2f}%)",
            f"{vals['TN']} ({vals['TN']/testExamplesNumber*100:.2f}%)",
            f"{vals['TP']+vals['TN']} ({(vals['TP']+vals['TN'])/testExamplesNumber*100:.2f}%)",
            f"{vals['FP']} ({vals['FP']/testExamplesNumber*100:.2f}%)",
            f"{vals['FN']} ({vals['FN']/testExamplesNumber*100:.2f}%)",
            f"{vals['FP']+vals['FN']} ({(vals['FP']+vals['FN'])/testExamplesNumber*100:.2f}%)",
            vals['FP']+vals['FN']+vals['TP']+vals['TN']
        ]
        for _, vals in confusionTFPN.items()
    ]
    columns = ['TP (%)', 'TN (%)', 'T (%)', 'FP (%)', 'FN (%)', 'F (%)', f"Total"]
    
    # Colorize cells depending on value
    cellColoursTF=plt.cm.Blues([
        [ (vals['TP']+vals['TN'])/testExamplesNumber, (vals['FP']+vals['FN'])/testExamplesNumber ] for _, vals in confusionTFPN.items()
    ])
    cellColourPos=plt.cm.Greens([
        [ vals['TP']/(vals['TP']+vals['FP']), vals['FP']/(vals['TP']+vals['FP']) ] for _, vals in confusionTFPN.items()
    ])
    cellColourNeg=plt.cm.Reds([
        [ vals['TN']/(vals['TN']+vals['FN']), vals['FN']/(vals['TN']+vals['FN']) ] for _, vals in confusionTFPN.items()
    ])
    
    cellColours = [
        [
            cellColourPos[i][0],
            cellColourNeg[i][0],
            cellColoursTF[i][0],
            cellColourPos[i][1],
            cellColourNeg[i][1],
            cellColoursTF[i][1],
            [0, 0, 0, 0]
        ]
        for i in range(len(rows))
    ]
        
    the_table = axs[axsLine].table(
      cellText=data,
      rowLabels=rows,
      colLabels=columns,
      loc='center',
      cellColours=cellColours
    )
    # the_table.scale(1.2, 1)
    the_table.auto_set_font_size(False)
    the_table.set_fontsize(20)
    axs[axsLine].axis('off')
    axs[axsLine].axis('tight')
    axs[axsLine].set_title(f"Confusion matrix {axsTitle}", fontsize=20, pad=0)    
    
    # TABLES / Confusion    
    # Output confusion matrix as plot table
    data = [[o for _,o in others.items()] for _,others in confusionLabels.items()]
    columns = rows
    
    # Colorize cells depending on value
    vals = [o for _,others in confusionLabels.items() for _,o in others.items()]
    normal = plt.Normalize(min(vals)-1, max(vals)+1)
    cellColours=plt.cm.Blues(normal(data))
        
    the_table = axs[axsLine+1].table(
      cellText=data,
      rowLabels=rows,
      colLabels=columns,
      loc='center',
      cellColours=cellColours
    )
    
    the_table.set_fontsize(20)
    # the_table.scale(1, 4)
    axs[axsLine+1].axis('off')
    axs[axsLine+1].axis('tight')
    axs[axsLine+1].set_title(f"True/Predicted {axsTitle}", fontsize=20)

In [ ]:
#### IMPORT LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#to load matlab mat files
from scipy.io import loadmat
# for saving metrics
import os 
import json
# for neural network
import tensorflow as tf
from tensorflow import keras

if not os.path.exists('./metrics'):
    os.makedirs('./metrics')

# FOR EACH LABEL GROUP
for labelNames in labelsGroup:
    # LOAD FILES
    print("\n\nLABELS", ', '.join(labelNames))

    # Load Training/Dev/Test data
    mat=loadmat(f"../datasets/train/{'_'.join(labelNames)}.mat")
    X, y = mat["X"], mat["y"]
    matDev=loadmat(f"../datasets/dev/{'_'.join(labelNames)}.mat")
    X_valid, y_valid = matDev["X"], matDev["y"]
    matTest=loadmat(f"../datasets/test/{'_'.join(labelNames)}.mat")
    X_test, y_test = matTest["X"], matTest["y"]
    
    # NUMBERS
    m = X.shape[0] # number of training examples
    labels = np.max(y)+1 # number of labels
    features = X.shape[1] # number of features per example

    # Output user feedback
    print(f"Loaded {m} traing examples with {labels} labels, each with {features} features (pixels).")
    print(f"Loaded {X_valid.shape[0]} dev examples.")
    print(f"Loaded {X_test.shape[0]} test examples.")
    
    print()
    
    # Create folder for metrics
    labelFolder = f"./metrics/{'_'.join(labelNames)}"
    if not os.path.exists(labelFolder):
        os.makedirs(labelFolder)
    if not os.path.exists(f"{labelFolder}/history"):
        os.makedirs(f"{labelFolder}/history")
    if not os.path.exists(f"{labelFolder}/predict"):
        os.makedirs(f"{labelFolder}/predict")
    if not os.path.exists(f"{labelFolder}/archive"):
        os.makedirs(f"{labelFolder}/archive")
    
    # FOREACH HIDDEN LAYER
    for hiddenLayers in hiddenLayersGroup:
        # FOR EACH HYPER PARAMETER
        for hyperParameter in hyperParametersGroup:
            # FOR EACH ITERATION NUMBER
            for iterations in iterationsGroup:
                
                hiddenLayersText = '_'.join(map(str, hiddenLayers)) if len(hiddenLayers)>1 else hiddenLayers[0]
                print(hiddenLayersText)
                combinationName = f"{iterations}iter_{batchSize}batchS_{hyperParameter}hyper_{len(hiddenLayers)}hlayers__{hiddenLayersText}"
                folder = f"{labelFolder}/archive/{combinationName}/"
                if not os.path.exists(folder):
                    os.makedirs(folder)

                print(f"\nBuilding network for {iterations} iterations and batch size of {batchSize} and {len(hiddenLayers)} hidden layers: {hiddenLayersText}...")
                print("With regularization!" if hyperParameter else "Without regularization!")
                print()

                # BUILD NETWORK
                # Create model
                modelSeq = []
                # Flattens each image (48x48) to 1x2304
                modelSeq.append(keras.layers.Flatten(input_shape = [2304]))
                # Hidden layers with relu activation function
                for h in hiddenLayers:                        
                    modelSeq.append(keras.layers.Dense(
                        h, 
                        activation = "relu", 
                        bias_regularizer= keras.regularizers.l2(hyperParameter) if hyperParameter else None
                    ))
                # Output layer with softmax activation function
                modelSeq.append(keras.layers.Dense(
                    labels, 
                    activation = "softmax",
                    bias_regularizer= keras.regularizers.l2(hyperParameter) if hyperParameter else None
                ))

                model = keras.models.Sequential(modelSeq)

                # Compile model
                model.compile(
                    # Using sparse categorical crossentropy loss function
                    loss = "sparse_categorical_crossentropy",
                    # Using stochastic gradient descent as gradient descent
                    optimizer = "sgd",
                    # In addition to cost, we want accuracy to help understanding how the model is working 
                    metrics = ["accuracy"]
                )

                # Train the network
                history = model.fit(
                    X,
                    y,
                    epochs = iterations,
                    batch_size = batchSize,
                    validation_data = (X_valid, y_valid)
                )

                # METRICS (SAVE TO FILE)

                # Model training history
                with open(f"{folder}/history.json", "w") as f:
                    json.dump(history.history, f)

                # Test model evaluation
                with open(f"{folder}/evaluation.json", "w") as f:
                    json.dump(model.evaluate(X_test, y_test, return_dict=True), f)

                # Prediction for test data
                with open(f"{folder}/predict.json", "w") as f:
                    json.dump(model.predict(X_test).tolist(), f)

                print(f"\nModel trained and metrics saved to {folder}!")

                # GENERATE GRAPHS AND SAVE TO FILE

                # Model training history
                pd.DataFrame(history.history).plot(figsize = (16, 10))
                plt.grid(True)
                plt.gca().set_ylim(0, 1)
                plt.title("Neural network training metrics")
                plt.savefig(f"{labelFolder}/history/{combinationName}.png")

                # Prediction for test data
                predict = model.predict(X_test)
                y_pred = np.array([np.argmax(p) for p in predict])
                confusionTFPN, confusionLabels = confusionMatrix(labelNames, y_pred, y_test)
                fig, axs = plt.subplots(2,1)
                outputConfusionTables(labelNames, y_test.size, confusionTFPN, confusionLabels, axs, 0, "WITHOUT regularization")
                fig.set_size_inches(18,6*len(labelNames))
                fig.subplots_adjust(left=0.2, top=20)
                fig.tight_layout()
                fig.savefig(f"{labelFolder}/predict/{combinationName}.png")



LABELS happy, neutral
Loaded 2000 traing examples with 2 labels, each with 2304 features (pixels).
Loaded 400 dev examples.
Loaded 400 test examples.

20

Building network for 200 iterations and batch size of 128 and 1 hidden layers: 20...
Without regularization!

Epoch 1/200
16/16 [==============================] - 0s 10ms/step - loss: 0.6849 - accuracy: 0.5513 - val_loss: 0.6745 - val_accuracy: 0.5825
Epoch 2/200
16/16 [==============================] - 0s 4ms/step - loss: 0.6699 - accuracy: 0.5867 - val_loss: 0.7086 - val_accuracy: 0.5400
Epoch 3/200
16/16 [==============================] - 0s 5ms/step - loss: 0.6730 - accuracy: 0.5780 - val_loss: 0.6651 - val_accuracy: 0.5900
Epoch 4/200
16/16 [==============================] - 0s 4ms/step - loss: 0.6701 - accuracy: 0.5802 - val_loss: 0.6729 - val_accuracy: 0.5650
Epoch 5/200
16/16 [==============================] - 0s 3ms/step - loss: 0.6719 - accuracy: 0.5589 - val_loss: 0.6705 - val_accuracy: 0.5875
Epoch 6/200
16/16 [========

Epoch 57/200
16/16 [==============================] - 0s 3ms/step - loss: 0.5640 - accuracy: 0.6910 - val_loss: 0.5640 - val_accuracy: 0.7150
Epoch 58/200
16/16 [==============================] - 0s 3ms/step - loss: 0.5690 - accuracy: 0.7060 - val_loss: 0.6612 - val_accuracy: 0.6200
Epoch 59/200
16/16 [==============================] - 0s 3ms/step - loss: 0.5757 - accuracy: 0.6786 - val_loss: 0.6759 - val_accuracy: 0.5975
Epoch 60/200
16/16 [==============================] - 0s 3ms/step - loss: 0.5728 - accuracy: 0.6906 - val_loss: 0.6559 - val_accuracy: 0.6400
Epoch 61/200
16/16 [==============================] - 0s 4ms/step - loss: 0.5658 - accuracy: 0.6978 - val_loss: 0.6773 - val_accuracy: 0.6250
Epoch 62/200
16/16 [==============================] - 0s 3ms/step - loss: 0.5632 - accuracy: 0.7064 - val_loss: 0.5652 - val_accuracy: 0.7200
Epoch 63/200
16/16 [==============================] - 0s 3ms/step - loss: 0.5316 - accuracy: 0.7315 - val_loss: 0.5722 - val_accuracy: 0.7025
Epoch 

16/16 [==============================] - 0s 3ms/step - loss: 0.5158 - accuracy: 0.7305 - val_loss: 0.5569 - val_accuracy: 0.6975
Epoch 115/200
16/16 [==============================] - 0s 3ms/step - loss: 0.4843 - accuracy: 0.7612 - val_loss: 0.6227 - val_accuracy: 0.6825
Epoch 116/200
16/16 [==============================] - 0s 3ms/step - loss: 0.5340 - accuracy: 0.7264 - val_loss: 0.5985 - val_accuracy: 0.7000
Epoch 117/200
16/16 [==============================] - 0s 4ms/step - loss: 0.4768 - accuracy: 0.7737 - val_loss: 0.5572 - val_accuracy: 0.7100
Epoch 118/200
16/16 [==============================] - 0s 4ms/step - loss: 0.4835 - accuracy: 0.7674 - val_loss: 0.5794 - val_accuracy: 0.6975
Epoch 119/200
16/16 [==============================] - 0s 4ms/step - loss: 0.4896 - accuracy: 0.7532 - val_loss: 0.6024 - val_accuracy: 0.7000
Epoch 120/200
16/16 [==============================] - 0s 3ms/step - loss: 0.4940 - accuracy: 0.7540 - val_loss: 0.5914 - val_accuracy: 0.7100
Epoch 121/200

16/16 [==============================] - 0s 5ms/step - loss: 0.4723 - accuracy: 0.7815 - val_loss: 0.5805 - val_accuracy: 0.6775
Epoch 172/200
16/16 [==============================] - 0s 5ms/step - loss: 0.5048 - accuracy: 0.7374 - val_loss: 0.5665 - val_accuracy: 0.7200
Epoch 173/200
16/16 [==============================] - 0s 5ms/step - loss: 0.4569 - accuracy: 0.7811 - val_loss: 0.5800 - val_accuracy: 0.6775
Epoch 174/200
16/16 [==============================] - 0s 5ms/step - loss: 0.4525 - accuracy: 0.7915 - val_loss: 0.6108 - val_accuracy: 0.7225
Epoch 175/200
16/16 [==============================] - 0s 5ms/step - loss: 0.4338 - accuracy: 0.7988 - val_loss: 0.5615 - val_accuracy: 0.7075
Epoch 176/200
16/16 [==============================] - 0s 5ms/step - loss: 0.4477 - accuracy: 0.7830 - val_loss: 0.6100 - val_accuracy: 0.7050
Epoch 177/200
16/16 [==============================] - 0s 6ms/step - loss: 0.4493 - accuracy: 0.7888 - val_loss: 0.6354 - val_accuracy: 0.6950
Epoch 178/200

16/16 [==============================] - 0s 6ms/step - loss: 0.5944 - accuracy: 0.6803 - val_loss: 0.6081 - val_accuracy: 0.6850
Epoch 27/200
16/16 [==============================] - 0s 6ms/step - loss: 0.5923 - accuracy: 0.7014 - val_loss: 0.6063 - val_accuracy: 0.6850
Epoch 28/200
16/16 [==============================] - 0s 6ms/step - loss: 0.6240 - accuracy: 0.6630 - val_loss: 0.5984 - val_accuracy: 0.7050
Epoch 29/200
16/16 [==============================] - 0s 6ms/step - loss: 0.5968 - accuracy: 0.6790 - val_loss: 0.6057 - val_accuracy: 0.6800
Epoch 30/200
16/16 [==============================] - 0s 6ms/step - loss: 0.6068 - accuracy: 0.6672 - val_loss: 0.6090 - val_accuracy: 0.6850
Epoch 31/200
16/16 [==============================] - 0s 6ms/step - loss: 0.5739 - accuracy: 0.7365 - val_loss: 0.6158 - val_accuracy: 0.6575
Epoch 32/200
16/16 [==============================] - 0s 6ms/step - loss: 0.5907 - accuracy: 0.6990 - val_loss: 0.6125 - val_accuracy: 0.6600
Epoch 33/200
16/16 